### Melon 100 챠트 스크래핑
1. 100곡의 노래의 제목과 ID 추출 (정규표현식)
2. 노래의 ID로 상세페이지를 100번 요청해서 상세정보를 추출
3. Json file에 저장
4. json file을 DataFrame 객체로 생성해서 표데이터 만들기
5. 표데이터를 DB에 저장하기

In [20]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
# 요청헤더 
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)

if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    # <div id="tb_list"> 
    print(len(soup.select('div#tb_list'))) #1
    # <div id="tb_list">  하위의 tr 선택
    print(len(soup.select('div#tb_list tr'))) #101
    # <div id="tb_list">  하위의 tr 하위의 <div class="wrap_song_info">
    print(len(soup.select('div#tb_list tr div.wrap_song_info'))) #200
    # <div id="tb_list">  하위의 tr 하위의 <div class="wrap_song_info"> 하위의 <a href="playSong">
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100
    
    song_atag = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    # 100개의 song정보를 저장할 리스트 선언
    song_list = []
    for song in song_atag:
        # 1개의 song 정보를 저장한 딕셔너리 선언
        song_dict = {}
        title = song.text
        href_value = song['href']
        matched = re.search(r'(\d+)\);', href_value)
        #print(matched.group(0), matched.group(1))
        song_id = matched.group(1)
        song_detail_link = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        print(title, song_detail_link)

200
1
101
200
100
롤린 (Rollin') https://www.melon.com/song/detail.htm?songId=30287019
라일락 https://www.melon.com/song/detail.htm?songId=33372781
Peaches (Feat. Daniel Caesar & Giveon) https://www.melon.com/song/detail.htm?songId=33359317
Celebrity https://www.melon.com/song/detail.htm?songId=33239419
운전만해 (We Ride) https://www.melon.com/song/detail.htm?songId=32860397
Timeless https://www.melon.com/song/detail.htm?songId=1944399
라라라 https://www.melon.com/song/detail.htm?songId=1854856
내사람 https://www.melon.com/song/detail.htm?songId=1500196
Coin https://www.melon.com/song/detail.htm?songId=33372783
내 손을 잡아 https://www.melon.com/song/detail.htm?songId=3414749
Dynamite https://www.melon.com/song/detail.htm?songId=32872978
On The Ground https://www.melon.com/song/detail.htm?songId=33346446
LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지) https://www.melon.com/song/detail.htm?songId=33331004
밝게 빛나는 별이 되어 비춰줄게 https://www.melon.com/song/detail.htm?songId=33167063
밤하늘의 별을(2020) https://www.melon.com/song